In [2]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 40.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 4.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.9 MB/s eta 0:00:000m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 KB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 KB 56.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tqdm import trange
from itertools import filterfalse
from tqdm import trange
from tqdm.notebook import tqdm
import random

# import contractions

import re
import time
import gc

In [2]:
#  select data whose topic is anxiety, depression and stress 
raw_data = pd.read_csv('data_counsel_chat_cleaned.csv')
options = ['anxiety', 'depression', 'stress'] 
chat_df = raw_data[raw_data['topic'].isin(options)] 

In [3]:
chat_df.tail()

,Unnamed: 0.1,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
581,1818,1818,860,How can I be less stressed?,I need help dealing with stress. How can I han...,https://counselchat.com/questions/how-can-i-be...,stress,Linda AbdelsayedChange is the only constant in...,https://counselchat.com/therapists/linda-abdel...,There are many causes for feeling stressed out...,0,25,train
582,1819,1819,861,How can I be less stressed?,I need help knowing how to deal with stress. W...,https://counselchat.com/questions/how-can-i-be...,stress,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,Something different works for each of us.There...,0,97,train
583,1820,1820,861,How can I be less stressed?,I need help knowing how to deal with stress. W...,https://counselchat.com/questions/how-can-i-be...,stress,"Kim HollingdaleMind-Body Approaches to Stress,...",https://counselchat.com/therapists/kim-holling...,Our body reacts to stress typically by breathi...,0,75,train
584,1821,1821,861,How can I be less stressed?,I need help knowing how to deal with stress. W...,https://counselchat.com/questions/how-can-i-be...,stress,Linda LawlessA Safe Place To Be,https://counselchat.com/therapists/linda-lawless,Learn how to meditate. I recommend a Mindful B...,0,45,train
585,1822,1822,862,How do I relieve stress and find balance?,"I'm in my late 20s. I have two jobs right now,...",https://counselchat.com/questions/how-do-i-rel...,stress,Richie (Yerachmiel) DonowitzExperienced - Comp...,https://counselchat.com/therapists/richie-yera...,A good start is to pay attention to some basic...,0,49,train


In [4]:
chat_df.shape

(586, 13)

In [5]:
chat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0.1   586 non-null    int64 
 1   Unnamed: 0     586 non-null    int64 
 2   questionID     586 non-null    int64 
 3   questionTitle  586 non-null    object
 4   questionText   586 non-null    object
 5   questionLink   586 non-null    object
 6   topic          586 non-null    object
 7   therapistInfo  586 non-null    object
 8   therapistURL   586 non-null    object
 9   answerText     586 non-null    object
 10  upvotes        586 non-null    int64 
 11  views          586 non-null    int64 
 12  split          586 non-null    object
dtypes: int64(5), object(8)
memory usage: 59.6+ KB


In [6]:
import torch
from transformers import pipeline

generate_text = \
pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

In [7]:
def dolly_generate_response(question_text):
    generated_text_obj = generate_text(question_text)
    response = generated_text_obj[0]["generated_text"]
    return response

In [8]:
# generate and save dolly's answers to question texts. 
from tqdm import tqdm

tqdm.pandas(desc="generate dolly answers")
chat_df['dollyAnswerText'] = chat_df['questionText'].progress_apply(dolly_generate_response)

generate dolly answers:   2%|██▋                                                                                                                                          | 11/586 [01:06<36:26,  3.80s/it]/home/simon/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
generate dolly answers: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 586/586 [48:05<00:00,  4.92s/it]


In [11]:
chat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0.1     586 non-null    int64 
 1   Unnamed: 0       586 non-null    int64 
 2   questionID       586 non-null    int64 
 3   questionTitle    586 non-null    object
 4   questionText     586 non-null    object
 5   questionLink     586 non-null    object
 6   topic            586 non-null    object
 7   therapistInfo    586 non-null    object
 8   therapistURL     586 non-null    object
 9   answerText       586 non-null    object
 10  upvotes          586 non-null    int64 
 11  views            586 non-null    int64 
 12  split            586 non-null    object
 13  dollyAnswerText  586 non-null    object
dtypes: int64(5), object(9)
memory usage: 64.2+ KB


In [12]:
chat_df.to_csv("data_counsel_chat_with_dolly_answers.csv")